In [1]:
import numpy as np
import os
from subprocess import Popen, PIPE, call
import subprocess
import sys
import time
from fnmatch import fnmatch
import h5py
sys.path.append( str(os.getcwd()) + '/src/scripts/')
import paths
import init_values as In

In [1]:
#imports from FitComparison_3panelPlot
import os
import h5py as h5
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import astropy.units as u
from astropy import constants as const

from scipy import interpolate
from scipy.optimize import minimize
from scipy.optimize import curve_fit

#from astropy.cosmology import WMAP9, z_at_value
from astropy.cosmology import Planck15  as cosmo# Planck 2018
from astropy.cosmology import z_at_value
from astropy.table import Table

from matplotlib import rc
import matplotlib
from matplotlib import ticker, cm
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib.colors import ListedColormap

############################
# Custom scripts
import get_ZdepSFRD as Z_SFRD

In [2]:
def Call_Cosmic_Integration(root_out_dir, COMPASfilename, rate_file_name, user_email, jname = 'CI',
                            ZdepSFRD_param_sets = [[],[]], #[[fid_dpdZ_parameters, fid_sfr_parameters]],
                           partitions = 'conroy,hernquist,shared', Wtime = "5:00:00", mem = "150000",
                           number_of_nodes = 1, number_of_cores = 1):
    if not os.path.isdir(root_out_dir+'/RateData/'):
        print('make output dir for rate data: ', root_out_dir+'/RateData/')
        os.mkdir(root_out_dir+'/RateData/')

    # Index to name your job
    n_CI = 1
    check_job_completionID = []

    # Run over each complete variation of the metallicity dependent SFRD
    for SFRD_zZ in ZdepSFRD_param_sets:

        print(10* "*" + ' You are Going to Run FastCosmicIntegration.py')
        print('dPdZ data = ', SFRD_zZ[0])
        print('SFR(z) data = ', SFRD_zZ[1])

        mu0, muz, sigma0, sigmaz, alpha0 = SFRD_zZ[0]
        sf_a, sf_b, sf_c, sf_d           = SFRD_zZ[1]
        DEPEND, append_job_id = False, 0

        # Flag to pass to FasCosmicIntegrator   #'/RateData/'+str(n_CI)+'_'+rate_file_name+\
        Flags = " --path "+root_out_dir + " --filename "+ COMPASfilename+" --outfname " +'/n/holystore01/LABS/hernquist_lab/Users/lvanson/home_output/SFRD_fit/src/data/RateData/'+str(n_CI)+'_'+rate_file_name+\
        " --mu0 " +str(mu0)+" --muz "+str(muz)+" --sigma0 "+str(sigma0)+" --sigmaz "+str(sigmaz)+" --alpha "+str(alpha0)+\
        " --aSF " +str(sf_a)+" --bSF "+str(sf_b)+" --cSF "+str(sf_c)+" --dSF "+str(sf_d)+\
        " --weight "+"mixture_weight"+ " --zstep "+"0.01"+" --sens "+"O3"+ " --m1min "+"5."+ " --dco_type BBH"+ " --BinAppend"+ \
        " --redshiftBinSize "+"0.05" + ' --maxzdet ' + "0.5" #+ " --BinAppend "
        
        run_dir = In.script_dir +'/CosmicIntegration/'

        check_job_completionID.append(CIjob_id.decode("utf-8"))

        n_CI += 1
        DEPEND, append_job_id = False, CIjob_id # no need for it to be dependent


    np.savetxt(root_out_dir+'/RateData/CI_job_IDs.txt', np.c_[check_job_completionID],header = "# job IDs ", delimiter=',', fmt="%s")

In [3]:
Zsun = 0.014 #solar metallicity
mu0_best      =0.025
muz_best      =-0.049
sigma0_best   =1.129
sigmaz_best   =0.048
alpha0_best  =-1.778

sf_a_best     =0.017
sf_b_best     =1.481
sf_c_best     =4.452
sf_d_best    =5.913
fid_dpdZ_parameters = [mu0_best, muz_best, sigma0_best, sigmaz_best, alpha0_best]
fid_sfr_parameters  = [sf_a_best, sf_b_best, sf_c_best, sf_d_best]

In [4]:
mu0, muz, sigma0, sigmaz, alpha0 = fid_dpdZ_parameters
sf_a, sf_b, sf_c, sf_d           = fid_sfr_parameters
DEPEND, append_job_id = False, 0
COMPASfilename = 'COMPAS_Output_wWeights.h5'
root_out_dir = '/home/sashalvna/research/Fit_SFRD_TNG100/'
rate_file_name = 'Rate_info_test.h5'

In [5]:
Flags = " --path "+root_out_dir + " --filename "+ COMPASfilename+ " --outfname " +'/home/sashalvna/research/Fit_SFRD_TNG100/' \
        +'_'+rate_file_name+ " --mu0 " +str(mu0)+" --muz "+str(muz)+" --sigma0 "+str(sigma0)+" --sigmaz "+str(sigmaz)+" --alpha "+str(alpha0)+\
        " --aSF " +str(sf_a)+" --bSF "+str(sf_b)+" --cSF "+str(sf_c)+" --dSF "+str(sf_d)+\
        " --weight "+"mixture_weight"+ " --zstep "+"0.01"+" --sens "+"O3"+ " --m1min "+"5."+ " --dco_type BBH"+ " --dontAppend"+ \
        " --redshiftBinSize "+"0.05" + ' --maxzdet ' + "0.5" 

In [ ]:
!python FastCosmicIntegration.py \
 --path /home/sashalvna/research/Fit_SFRD_TNG100/ --filename COMPAS_Output_wWeights.h5 --outfname /home/sashalvna/research/Fit_SFRD_TNG100/_Rate_info_test.h5 --mu0 0.025 --muz -0.049 --sigma0 1.129 --sigmaz 0.048 --alpha -1.778 --aSF 0.017 --bSF 1.481 --cSF 4.452 --dSF 5.913 --weight mixture_weight --zstep 0.01 --sens O3 --m1min 5. --dco_type BBH --redshiftBinSize 0.05 --maxzdet 0.5

In [6]:
Flags

' --path /home/sashalvna/research/Fit_SFRD_TNG100/ --filename COMPAS_Output_wWeights.h5 --outfname /home/sashalvna/research/Fit_SFRD_TNG100/_Rate_info_test.h5 --mu0 0.025 --muz -0.049 --sigma0 1.129 --sigmaz 0.048 --alpha -1.778 --aSF 0.017 --bSF 1.481 --cSF 4.452 --dSF 5.913 --weight mixture_weight --zstep 0.01 --sens O3 --m1min 5. --dco_type BBH --dontAppend --redshiftBinSize 0.05 --maxzdet 0.5'

In [17]:
import h5py as h5
f = '/home/sashalvna/research/Fit_SFRD_TNG100/COMPAS_Output_wWeights.h5'
data = h5.File(f, 'r')
list(data.keys())

['BSE_Double_Compact_Objects',
 'BSE_RLOF',
 'BSE_System_Parameters',
 'Rates_mu00.025_muz-0.05_alpha-1.77_sigma01.125_sigmaz0.05_a0.02_b1.48_c4.45_d5.9_zBinned']

In [33]:
!python FastCosmicIntegration.py \
 --path /home/sashalvna/research/Fit_SFRD_TNG100/ --filename COMPAS_Output_wWeights_reduced.h5 \
 --outfname /home/sashalvna/research/Fit_SFRD_TNG100/Rate_info_test.h5 --mu0 0.025 --muz -0.049 \
 --sigma0 1.129 --sigmaz 0.048 --alpha -1.778 --aSF 0.017 --bSF 1.481 --cSF 4.452 --dSF 5.913 \
 --weight mixture_weight --zstep 0.01 --sens O3 --m1min 5. --dco_type BBH  --BinAppend \
 --redshiftBinSize 0.05 --maxzdet 0.5 --cosmology Planck15

Using Planck15 as cosmology!
Calculate detection rates
Running find_detection_rate with the following parameters: path /home/sashalvna/research/Fit_SFRD_TNG100/, filename COMPAS_Output_wWeights_reduced.h5, dco_typeBBH, weight_column mixture_weight
merges_hubble_time True, pessimistic_CEE True, no_RLOF_after_CE True,
max_redshift 10, max_redshift_detection 0.5, redshift_step 0.01, z_first_SF 10,
m1_min 5.0 solMass, m1_max 150.0 solMass, m2_min 0.1 solMass, fbin 0.7,
aSF 0.017, bSF 1.481, cSF 4.452, dSF 5.913, mu0 0.025, muz -0.049, sigma0 1.129, sigmaz 0.048, alpha -1.778, min_logZ -12.0, max_logZ 0.0, step_logZ 0.01
sensitivity O3, snr_threshold 8, Mc_max 300.0, Mc_step 0.1, eta_max 0.25, eta_step 0.01, snr_max 1000.0, snr_step 0.1
noRLOFafterCEE, pessimistic: True True
sum(type_masks[BBH]) 898344, sum(hubble_mask) 713872, sum(rlof_mask) 1000000, sum(pessimistic_mask) 1000000
/home/sashalvna/research/SFRD_fit/src/scripts/CosmicIntegration/FastCosmicIntegration.py:859: UserWarning: Maxi

In [34]:
!python FastCosmicIntegration.py \
 --path /home/sashalvna/research/Fit_SFRD_TNG100/ --filename COMPAS_Output_wWeights.h5 \
 --outfname /home/sashalvna/research/Fit_SFRD_TNG100/Rate_info_test.h5 --mu0 0.025 --muz -0.049 \
 --sigma0 1.129 --sigmaz 0.048 --alpha -1.778 --aSF 0.017 --bSF 1.481 --cSF 4.452 --dSF 5.913 \
 --weight mixture_weight --zstep 0.01 --sens O3 --m1min 5. --dco_type BBH  --BinAppend \
 --redshiftBinSize 0.05 --maxzdet 0.5 --cosmology Planck15

Using Planck15 as cosmology!
Calculate detection rates
Running find_detection_rate with the following parameters: path /home/sashalvna/research/Fit_SFRD_TNG100/, filename COMPAS_Output_wWeights.h5, dco_typeBBH, weight_column mixture_weight
merges_hubble_time True, pessimistic_CEE True, no_RLOF_after_CE True,
max_redshift 10, max_redshift_detection 0.5, redshift_step 0.01, z_first_SF 10,
m1_min 5.0 solMass, m1_max 150.0 solMass, m2_min 0.1 solMass, fbin 0.7,
aSF 0.017, bSF 1.481, cSF 4.452, dSF 5.913, mu0 0.025, muz -0.049, sigma0 1.129, sigmaz 0.048, alpha -1.778, min_logZ -12.0, max_logZ 0.0, step_logZ 0.01
sensitivity O3, snr_threshold 8, Mc_max 300.0, Mc_step 0.1, eta_max 0.25, eta_step 0.01, snr_max 1000.0, snr_step 0.1
noRLOFafterCEE, pessimistic: True True
sum(type_masks[BBH]) 2273376, sum(hubble_mask) 1835145, sum(rlof_mask) 2523122, sum(pessimistic_mask) 2523122
/home/sashalvna/research/SFRD_fit/src/scripts/CosmicIntegration/FastCosmicIntegration.py:859: UserWarning: Maximum ch

In [26]:
! python /home/sashalvna/research/Fit_SFRD_TNG100/reduceHdf5Data.py \
/home/sashalvna/research/Fit_SFRD_TNG100/COMPAS_Output_wWeights.h5 \
/home/sashalvna/research/Fit_SFRD_TNG100/COMPAS_Output_wWeights_reduced.h5

The main files I have at my disposal are:
 ['BSE_Double_Compact_Objects', 'BSE_RLOF', 'BSE_System_Parameters', 'Rates_mu00.025_muz-0.05_alpha-1.77_sigma01.125_sigmaz0.05_a0.02_b1.48_c4.45_d5.9_zBinned']


In [24]:
len(data['BSE_Double_Compact_Objects']['SEED'][()])

2523122

In [5]:
!python /home/sashalvna/research/COMPAS/compas_python_utils/cosmic_integration/FastCosmicIntegration.py \
 --path /home/sashalvna/research/Fit_SFRD_TNG100/COMPAS_Output_wWeights_reduced.h5 --mu0 0.025 --muz -0.049 \
 --sigma0 1.129 --sigmaz 0.048 --alpha -1.778 --aSF 0.017 --bSF 1.481 --cSF 4.452 --dSF 5.913 \
 --weight mixture_weight --zstep 0.01 --sens O3 --m1min 5. --dco_type BBH --maxzdet 0.5 --cosmology Planck15

Using Planck15 as cosmology!
Traceback (most recent call last):
  File "/home/sashalvna/research/COMPAS/compas_python_utils/cosmic_integration/FastCosmicIntegration.py", line 892, in <module>
    main()
  File "/home/sashalvna/research/COMPAS/compas_python_utils/cosmic_integration/FastCosmicIntegration.py", line 829, in main
    detection_rate, formation_rate, merger_rate, redshifts, COMPAS = find_detection_rate(
  File "/home/sashalvna/research/COMPAS/compas_python_utils/cosmic_integration/FastCosmicIntegration.py", line 420, in find_detection_rate
    COMPAS.setCOMPASDCOmask(types=dco_type, withinHubbleTime=merges_hubble_time, pessimistic=pessimistic_CEE, noRLOFafterCEE=no_RLOF_after_CEE)
  File "/home/sashalvna/research/COMPAS/compas_python_utils/cosmic_integration/ClassCOMPAS.py", line 102, in setCOMPASDCOmask
    ce_seeds = self.get_COMPAS_variables("BSE_Common_Envelopes", "SEED")
  File "/home/sashalvna/research/COMPAS/compas_python_utils/cosmic_integration/ClassCOMPAS.py", line 

In [2]:
import matplotlib
matplotlib.use('Agg')

import seaborn as sns
from scipy import stats

# Chosen cosmology - now it's WMAP9??
# Update: plot mass distribution.py doesnt use cosmo or z_at_value, so can ignore here
#from astropy.cosmology import WMAP9 as cosmo
#from astropy.cosmology import z_at_value

import json
import argparse
import gc

# Custom scripts
import MassDistHelperFunctions as mfunc
import Plot_Mass_distributions as pltmass

In [3]:
## locations
save_loc    =  '/home/sashalvna/research/Fit_SFRD_TNG100'
data_dir    =  '/home/sashalvna/research/Fit_SFRD_TNG100'
rate_file_name = '1_Rate_info.h5'
COMPASfilename = 'COMPAS_Output_wWeights.h5'

In [18]:
path = data_dir + '/' + rate_file_name
file = h5.File(path, 'r')
list(file.keys())

['Rates_mu00.025_muz-0.049_alpha-1.778_sigma01.125_sigmaz0.048_a0.02_b1.48_c4.44_d5.9_zBinned']

In [5]:
#figure 3: stable MT, no unstable so no CE
only_stable = True 
only_CE     = False 
channel_string = 'stable'

In [4]:
## PLOT setttings
import matplotlib.pyplot as plt
plt.rc('font', family='serif')
from matplotlib import rc

matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=False)
fsize, SMALL_SIZE, MEDIUM_SIZE, BIGGER_SIZE = 30,25,25,30
for obj in ['axes','xtick','ytick']:
    plt.rc(obj, labelsize=MEDIUM_SIZE)          # controls default text sizes
for obj in ['figure','axes']:
    plt.rc(obj, titlesize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize

In [6]:
#plotting
fig = plt.figure( figsize = (24, 28))

# width of SFRD at z=0 - sigma0 varied
subplot1 = fig.add_subplot(321) #first subplot in layout that has 3 rows and 2 columns
ax1 = pltmass.plot_mass_distribution(sim_dir = data_dir, data_dir=data_dir, rate_file='/'+str(rate_file_name), simulation_data = '/'+str(COMPASfilename),
                           x_key = 'M_moreMassive',  rate_keys  = ['Rates_mu00.025_muz-0.049_alpha-1.778_sigma01.125_sigmaz0.048_a0.02_b1.48_c4.44_d5.9_zBinned'], channel_string = channel_string,
                           show_hist = False, show_KDE = True,  plot_LIGO = True, Color =  'navy',
                           only_CE = only_CE, only_stable = only_stable, 
                           bootstrap = False, bootstraps = 50, save_name = 'SFRD_width_variations.pdf', titletext = "Width of metallicity dist."+"\n"+r"$\omega_0$, (scale $z=0$)",
                           labels = [r'$\mathrm{Narrow: \ }  (\omega_0 = 0.70) \  \mathcal{R}_{0.2} = \ $',
                                     r'$\mathrm{Fiducial: \ } (\omega_0 = 1.13) \ \mathcal{R}_{0.2}= \ $', 
                                     r'$\mathrm{Wide: \ } (\omega_0 = 2.00) \  \mathcal{R}_{0.2} = \ $'],
                          multipanel = True, subplot = subplot1)

plotting LIGO
Reading  /home/sashalvna/research/Fit_SFRD_TNG100//COMPAS_Output_wWeights.h5
Done with reading DCO data for this file :)
nplot 0 

rate_key Rates_mu00.025_muz-0.049_alpha-1.778_sigma01.125_sigmaz0.048_a0.02_b1.48_c4.44_d5.9_zBinned
using file with key CE_Event_Count
BBH_bool should already contain all stuff, np.sum(BBH_bool)  1640550  len(merging_BBH) 1640550
$\mathrm{Narrow: \ }  (\omega_0 = 0.70) \  \mathcal{R}_{0.2} = \ $  len(merging_BBH)= 449453  Rate =  19.829841847935224  Gpc-3 yr-1
get the histogram
m 1.4327445768535148 [1.43274458] [1.43274458]

m 0.48053673638125116 [0.48053674] [0.48053674]

m 0.023437340070893427 [0.02343734] [0.02343734]



In [1]:
! pwd

/home/sashalvna/research/SFRD_fit/src/scripts/CosmicIntegration


In [2]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

import seaborn as sns
from scipy import stats

# Chosen cosmology - now it's WMAP9??
# Update: plot mass distribution.py doesnt use cosmo or z_at_value, so can ignore here
#from astropy.cosmology import WMAP9 as cosmo
#from astropy.cosmology import z_at_value

import json
import argparse
import gc

# Custom scripts
import MassDistHelperFunctions as mfunc
import Plot_Mass_distributions as pltmass